# Datenhandling mit Pandas

Pandas ist ein Tool zur Verwaltung, Analyse und Manipulaton von Daten, die in tabellarischer Form vorliegen. Gute weitergehende Einführungen zu Pandas finden sich hier:

- https://pandas.pydata.org/pandas-docs/stable/getting_started/index.html
- https://jakevdp.github.io/PythonDataScienceHandbook/

### Bibliotheken importieren

Um auf die Klassen und Funktionen einer Bibliothek -- in Python `module` genannt -- zuzugreifen, müssen wir das module mit dem `import` Statement importieren. Durch den Zusatz `as pd` sagen wir Python, dass das module unter dem Kürzel `pd` im Workspace verfügbar gemacht werden soll. Auf den Inhalt kann man dann mit `pd.<Funktionsname>` bzw. `pd.<Klassenname>` zugreifen.

In [ ]:
import pandas as pd

### Daten einlesen
Die Daten liegen typischerweise in externen Dateien vor (etwa .csv oder .xls) und werden dann in einen Pandas DataFrame eingelesen.

In [ ]:
dfCountries = pd.read_csv('Daten/countries of the world.csv')

Oft müssen beim Einlesen noch einzelne Parameter eingestellt werden, wie in diesem Fall die Festlegung des Dezimalpunkts. Dafür ist es hilfreich mit dem Befehl help(pd.read_excel) die Documentation aufzurufen. csv-Dateien werden mit dem Befehl pd.read_csv() eingelesen.

### Daten sichten

Die Methode `.head` erlaubt einen ersten Blick in den Datensatz.

In [ ]:
dfCountries.head()

Praktisch: Die Hilfe zu einem Befehl kann man mittels `?` aufrufen:

In [ ]:
dfCountries.head?

Zufällige Datensätze werden mit dem Befehl .sample() angezeigt

In [ ]:
dfCountries.sample(10)

Grundlegende statistische Informationen zeigt der Befehl .describe(). 

In [ ]:
dfCountries.describe()

Die Datentypen der einzelnen Spalten, die Anzahl der Einträge (auch pro Spalte) sowie die Größe des Datensatzes im Speicher liefert der Befehl .info(). 

In [ ]:
dfCountries.info()

### Selektion einzelner Spalten

Einzelne Spalten können wie folgt gefiltert werden:

In [ ]:
dfCountries['Population']

Mehrere Spalten werden dabei als Liste übergeben:

In [ ]:
dfCountries[['Population','Birthrate','Deathrate']]

Dubletten können mit dem Befehl .unique() entfernt und angezeigt werden

In [ ]:
dfCountries['Region'].unique()

Eine Liste der Spalten ist so abrufbar:

In [ ]:
dfCountries.columns

# Filtern der Zeilen

Wir filtern nun alle Länder in der Region WESTERN EUROPE. Dies geht über das loc-Attribute.

In [ ]:
dfCountries.loc[dfCountries['Region']=='WESTERN EUROPE']

Ein zweites Beispiel

In [ ]:
dfCountries.loc[dfCountries['Population']>100000000]

Man kann Filter auch kombinieren. Dabei ist & das Symbol für eine Boolsche UND-Verknüpfung, | das Symbol für eine Boolsche ODER-Verknüpfung:

In [ ]:
dfCountries.loc[(dfCountries['Population']>100000000) & (dfCountries['Area (sq. mi.)']<1000000)]

Filter die häufig benutzt werden, können auch in eine sogenannte Mask ausgelagert werden

In [ ]:
maskGroßUndKlein=(dfCountries['Population']>100000000) | (dfCountries['Population']<10000) 

In [ ]:
dfCountries.loc[maskGroßUndKlein]

Es existiert auch folgende Kurzschreibweise

In [ ]:
dfCountries[dfCountries.Population<10000]

### Neue Spalten anlegen

Neue Spalten lassen sich durch eine einfache Zuweisung (zu einem bisher unbenutzen Spaltennamen) anlegen

In [ ]:
dfCountries['PopChange']=dfCountries['Birthrate']-dfCountries['Deathrate']

In [ ]:
dfCountries

Man kann dies auch mit Filtern kombinieren

In [ ]:
dfCountries.loc[dfCountries.Population<10000,'Winzig']=True

In [ ]:
dfCountries

In [ ]:
dfCountries[dfCountries.Winzig==True]

### Datenaggregation

Pandas DataFrames können mit dem .groupby-Befehl aggregiert werden. Danach stehen die üblichen Funktionen zur Auswertung zur Verfügung.

In [ ]:
dfCountries.groupby('Region').sum()

Möchte man individuelle Aggregationsfunktionen pro Spalte anwenden, kann man die Funktion .agg sowie ein Dictionary als Parameter benutzen:

In [ ]:
dfCountries.groupby('Region').agg({'Population':'sum', 'Climate':'count', 'Agriculture':'max'})

### Handling von fehlenden Daten

Mit dem Befehl .fillna können fehlende Werte durch Standardwerte ersetzt werden.

In [ ]:
dfCountries[['Service','Winzig']].head()

In [ ]:
dfCountries[['Winzig']].fillna(False)

Bei mehreren Spalten kann ein Dictionary für individelle Werte pro Spalte benutzt werden.

In [ ]:
dfCountries[['Service','Winzig']].fillna({'Service':0, 'Winzig':False})

Dabei ersetzt fillna die Werte nicht, sondern liefert nur eine angepasste Sicht. Wir überprüfen dies, indem wir den DataFrame nochmals betrachten

In [ ]:
dfCountries

Um die Werte auch im zugrundeliegenden Datensatz zu ändern, muss der Parameter inplace benutzt werden:

In [ ]:
dfCountries.fillna({'Service':0, 'Winzig':False}, inplace=True)

In [ ]:
dfCountries.head()

### DataFrames transponieren

Wir legen uns zuerst einen aggregierten, verkleinerten Datensatz zurecht:

In [ ]:
dfRegions=dfCountries[['Country','Region','Agriculture','Industry','Service']].groupby('Region').mean()

In [ ]:
dfRegions

Mit dem Befehl .T kann man einen DataFrame transponiert anzeigen

In [ ]:
dfRegions.T

### Verknüpfung (Join) auf verschiedenen Datensätzen

Wir speichern den DataFrame dfRegions in eine csv-Datei.

In [ ]:
dfRegions.to_csv('Daten/regions.csv')

Ein fleißiger Kollege hat für jede Region den zugehörigen Kontinent angefügt und das Ergebnis in der Datei 'regions_edited.csv' gespeichert. Wir laden die Datei in den DataFrame dfRegions

In [ ]:
dfRegions=pd.read_csv('Daten/regions_edited.csv')

In [ ]:
dfRegions

Unser Ziel ist nun, die DataFrames dfCountries und dfRegions so zu verknüpfen, dass wir für jedes Land den entsprechenden Kontinent als zusätzliche Spalte speichern.

In [ ]:
pd.merge(dfCountries,dfRegions, how='inner', left_on='Region', right_on='Region')

Die Spalten Industry, Agriculture und Service kommen doppelt vor, wir selektieren deswegen noch die entsprechenden Spalten

In [ ]:
dfCountries = pd.merge(dfCountries,dfRegions[['Region','Continent']], how='inner', left_on='Region', right_on='Region')

# ✏️ Übungsaufgaben

Zeigen Sie den Datensatz dfCountries so an, dass nur die Spalten 'Country', 'Agriculture', 'Industry' und 'Service' erscheinen.

Zeigen Sie den Datensatz dfCountries so an, dass nur die Spalte 'Net Migration' erscheint.

Zeigen Sie alle Länder an, deren Küstenlinie größer 30 ist.

Zeigen Sie 7 zufällige Länder an, deren Küstenlinie größer 30 ist.

Zeigen Sie alle Länder an, deren Küstenlinie größer 30 und kleiner 200 ist.

Fügen Sie zum DataFrame dfCountries eine Spalte an, bei der die Fläche in Quadratkilometern anstatt Quadratmeilen angegeben ist (1 Quadratmeile = 2,58999 Quadratkilometer).

Fügen Sie zum DataFrame dfCountries eine Spalte 'Increasing' an, deren Wert 'True' ist, wenn 'Birthrate'>'Deathrate' und 'False ' sonst.

Aggregieren Sie den DataFrame dfCountries. Gruppieren Sie dabei nach 'Continent'. Was ist die durchschnittliche Länderpopulation für jeden Kontinent. Was ist die Gesamtpopulation für jeden Kontinent? Wie groß ist das kleinste Land für jeden Kontinent?

Erstellen Sie im DataFrame dfCountries eine neue Spalte 'Migration Class' mit den Werten 'Small Migration' für 'Net Migration' < -5, 'Medium Migration' für 'Net Migration' zwischen -5 und 5 und 'High Migration' für 'Net Migration'>5.

Aggregieren Sie den DataFrame dfCountries. Groupieren Sie dabei nach der 'Migration Class'. Geben Sie für jede Klasse immer den kleinsten, durchschnittlichen und größten Wert der Spalte 'Net Migration' als eigene Spalte an.  Speichern Sie das Ergebnis in dem neuen DataFrame dfMigrationClasses.

Hinweis: Nach der Aggregation können Sie den Multilevel-Index mit "dfMigrationClasses.columns = dfMigrationClasses.columns.droplevel()" einebenen.

Filtern Sie dfCountries nach Kontinent 'ASIA', gruppieren dann nach 'Region'. Was ist der Median der 'Coastline' für jede Region in Asien?

Speichern Sie den DataFrame dfMigrationClasses als csv-Datei 'migclass.csv'.

Lesen Sie die Date 'migclass.csv' ein und speichern das Ergebnis in einem neuen DataFrame dfMigrationClasses2

Verknüpfen Sie die DataFrames dfCountries und dfMigrationClasses2. Zeigen Sie das Ergebnis an und speichern es in den DataFrame dfFinal.